In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

from google.colab import drive
drive.mount("/content/drive")

BASE_DIR = Path("/content/drive/MyDrive/bbref_drafts")  # change if your folder is different

OUTCOMES_PATH = BASE_DIR / "nba_outcomes_2010_2021_normalized.csv"
PER40_PATH    = BASE_DIR / "NCAA_per_game.csv"          # or your per-40 file name
ADV_PATH      = BASE_DIR / "NCAA_advanced.csv"

def read_csv_robust(fp: Path) -> pd.DataFrame:
    """
    Reads normal CSVs AND fixes the 'everything in one column' CSV case.
    Works without changing the file.
    """
    df = pd.read_csv(fp)

    # If it came in as one column and the column name contains commas, split it
    if df.shape[1] == 1 and isinstance(df.columns[0], str) and "," in df.columns[0]:
        raw = pd.read_csv(fp, header=None)
        split = raw[0].astype(str).str.split(",", expand=True)

        # First row becomes header
        split.columns = split.iloc[0].astype(str).str.strip()
        df = split.iloc[1:].reset_index(drop=True)

    # Strip whitespace from column names
    df.columns = [str(c).strip() for c in df.columns]
    return df

df_out = read_csv_robust(OUTCOMES_PATH)
df_p40 = read_csv_robust(PER40_PATH)
df_adv = read_csv_robust(ADV_PATH)

print("outcomes cols:", list(df_out.columns)[:12])
print("per40 cols:", list(df_p40.columns)[:12])
print("advanced cols:", list(df_adv.columns)[:12])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
outcomes cols: ['DraftYear', 'DraftPick', 'PlayerName', 'NCAA_Team', 'NBA_Yrs_Career', 'VORP_Career', 'VORP_Pctl_InClass', 'VORP_Z_InClass', 'VORP_Rating_0_100']
per40 cols: ['Rk', 'Player', 'Draft Team', 'Round', 'Pick', 'Draft Year▲', 'Draft College', 'Season', 'Team', 'G', 'GS', 'FG']
advanced cols: ['Rk', 'Player', 'Draft Team', 'Round', 'Pick', 'Draft Year', 'Draft College', 'Season', 'Team', 'G', 'GS', 'WS']


In [ ]:
def normalize_name(s):
    if pd.isna(s):
        return ""
    s = str(s).lower().strip()
    s = re.sub(r"[^a-z\s]", "", s)
    s = re.sub(r"\s+", " ", s)
    return s

# Outcomes file uses PlayerName
df_out["name_key"] = df_out["PlayerName"].apply(normalize_name)

# NCAA files: detect which column holds player name
def get_player_col(df):
    for c in ["Player", "PlayerName", "player", "name"]:
        if c in df.columns:
            return c
    raise KeyError(f"No player name column found. Columns: {df.columns.tolist()}")

p40_player_col = get_player_col(df_p40)
adv_player_col = get_player_col(df_adv)

df_p40["name_key"] = df_p40[p40_player_col].apply(normalize_name)
df_adv["name_key"] = df_adv[adv_player_col].apply(normalize_name)

print("Player column in per40:", p40_player_col)
print("Player column in advanced:", adv_player_col)


Player column in per40: Player
Player column in advanced: Player


In [ ]:
import re
import numpy as np
import pandas as pd

def season_end_year(season_val):
    """
    Converts '2009-10' -> 2010 (end year).
    Handles '2010-11', etc.
    """
    if pd.isna(season_val):
        return np.nan
    s = str(season_val).strip()
    m = re.match(r"^(\d{4})-(\d{2})$", s)
    if not m:
        return np.nan
    start = int(m.group(1))
    return start + 1

# Make sure DraftYear numeric
df_out["DraftYear"] = pd.to_numeric(df_out["DraftYear"], errors="coerce")

# Add SeasonEndYear to NCAA data
df_p40["SeasonEndYear"] = df_p40["Season"].apply(season_end_year)
df_adv["SeasonEndYear"] = df_adv["Season"].apply(season_end_year)

print("Per40 SeasonEndYear missing:", df_p40["SeasonEndYear"].isna().sum())
print("Adv SeasonEndYear missing:", df_adv["SeasonEndYear"].isna().sum())


Per40 SeasonEndYear missing: 0
Adv SeasonEndYear missing: 0


In [ ]:
# Only keep NCAA rows whose season end year matches the player's DraftYear
# We'll merge by (name_key, DraftYear) <-> (name_key, SeasonEndYear)

out_keys = df_out[["name_key", "DraftYear"]].dropna().drop_duplicates()

p40_final = df_p40.merge(
    out_keys,
    left_on=["name_key", "SeasonEndYear"],
    right_on=["name_key", "DraftYear"],
    how="inner"
)

adv_final = df_adv.merge(
    out_keys,
    left_on=["name_key", "SeasonEndYear"],
    right_on=["name_key", "DraftYear"],
    how="inner"
)

print("Per40 matched rows:", len(p40_final))
print("Adv matched rows:", len(adv_final))

# If a player appears multiple times (rare), keep the row with the most minutes if MP exists
def keep_max_mp(df):
    if "MP" in df.columns:
        df = df.copy()
        df["MP"] = pd.to_numeric(df["MP"], errors="coerce").fillna(0)
        df = df.sort_values(["name_key", "DraftYear", "MP"], ascending=[True, True, False])
        return df.drop_duplicates(subset=["name_key", "DraftYear"], keep="first")
    return df.drop_duplicates(subset=["name_key", "DraftYear"], keep="first")

p40_final = keep_max_mp(p40_final)
adv_final = keep_max_mp(adv_final)

print("Per40 after dedupe:", len(p40_final))
print("Adv after dedupe:", len(adv_final))


Per40 matched rows: 268
Adv matched rows: 268
Per40 after dedupe: 268
Adv after dedupe: 268


In [ ]:
ncaa_features = pd.merge(
    p40_final,
    adv_final,
    on=["name_key", "DraftYear"],
    how="inner",
    suffixes=("_p40", "_adv")
)

print("NCAA features rows:", len(ncaa_features))
display(ncaa_features[["name_key","DraftYear","Season_p40","Season_adv"]].head())


NCAA features rows: 268


,name_key,DraftYear,Season_p40,Season_adv
0,xavier henry,2010,2009-10,2009-10
1,trevor booker,2010,2009-10,2009-10
2,quincy pondexter,2010,2009-10,2009-10
3,paul george,2010,2009-10,2009-10
4,patrick patterson,2010,2009-10,2009-10


In [ ]:
df_model = pd.merge(
    df_out,
    ncaa_features,
    on=["name_key", "DraftYear"],
    how="inner"
)

print("Final model rows:", len(df_model))
display(df_model[["PlayerName","DraftYear","VORP_Pctl_InClass","VORP_Career"]].head(15))


Final model rows: 268


,PlayerName,DraftYear,VORP_Pctl_InClass,VORP_Career
0,John Wall,2010,0.965517,24.1
1,Evan Turner,2010,0.413793,0.5
2,Derrick Favors,2010,0.793103,14.0
3,DeMarcus Cousins,2010,0.896552,21.6
4,Ekpe Udoh,2010,0.448276,0.8
5,Greg Monroe,2010,0.827586,15.3
6,Al-Farouq Aminu,2010,0.724138,6.5
7,Gordon Hayward,2010,0.931034,23.0
8,Paul George,2010,1.000000,44.5
9,Cole Aldrich,2010,0.603448,1.8


In [ ]:
TARGET = "VORP_Pctl_InClass"

FEATURES = [
    "PTS",
    "TRB",
    "AST",
    "STL",
    "BLK",
    "TOV",
    "TS%",
    "TRB%",
    "AST%",
    "STL%",
    "BLK%",
    "TOV%",
    "USG%",
]

# Keep only what exists (safety)
FEATURES = [c for c in FEATURES if c in df_model.columns]

df_model_final = df_model[FEATURES + [TARGET]].copy()

# Convert to numeric and drop missing
for c in FEATURES + [TARGET]:
    df_model_final[c] = pd.to_numeric(df_model_final[c], errors="coerce")

df_model_final = df_model_final.dropna()

print("Final modeling rows:", len(df_model_final))
print("Features used:", FEATURES)
display(df_model_final.head(10))


Final modeling rows: 268
Features used: ['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']


,PTS,TRB,AST,STL,BLK,TOV,TS%,TRB%,AST%,STL%,BLK%,TOV%,USG%,VORP_Pctl_InClass
0,19.1,4.9,7.5,2.0,0.6,4.6,0.562,6.6,34.8,2.9,1.5,21.4,25.7,0.965517
1,22.8,10.3,6.7,1.9,1.0,4.9,0.581,15.9,37.5,3.0,2.9,20.0,33.1,0.413793
2,18.1,12.3,1.5,1.3,3.0,3.6,0.621,16.5,7.9,1.8,7.9,20.0,21.8,0.793103
3,25.8,16.8,1.7,1.7,3.0,3.5,0.579,22.5,9.2,2.4,7.5,13.6,30.5,0.896552
4,15.8,11.1,3.1,0.9,4.2,2.8,0.535,15.6,14.4,1.4,11.0,15.7,22.1,0.448276
5,18.9,11.3,4.4,1.4,1.8,3.8,0.567,17.5,22.4,2.2,4.9,18.8,26.7,0.827586
6,20.2,13.7,1.7,1.8,1.8,4.1,0.524,17.6,9.1,2.6,4.5,17.4,27.5,0.724138
7,18.5,9.8,2.0,1.3,1.0,2.7,0.601,15.8,12.1,2.0,2.8,15.2,24.7,0.931034
8,20.2,8.7,3.7,2.7,1.0,3.9,0.572,12.8,22.4,4.0,2.9,18.1,27.9,1.000000
9,16.9,14.7,1.3,1.1,5.2,2.3,0.596,20.0,5.8,1.6,13.0,14.1,19.9,0.603448


In [ ]:
from pathlib import Path

OUT_PATH = BASE_DIR / "model_data/ncaa_to_nba_model_data.csv"
OUT_PATH.parent.mkdir(exist_ok=True)

df_model_final.to_csv(OUT_PATH, index=False)
print("Saved:", OUT_PATH)


Saved: /content/drive/MyDrive/bbref_drafts/model_data/ncaa_to_nba_model_data.csv
